In [ ]:
## Import libs
import findspark
findspark.init("/home/antonis/spark-2.3.0-bin-hadoop2.7")

import os.path
import pandas
import math
import time

from metrics.Correlation import Correlation
from IO.Output import Output
from IO.Input import Input
from metrics.RSquare import RSquare
from DataSet.Dataset import DataSet

import pyspark
from pyspark.sql import SparkSession


import sys
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import countDistinct,avg,stddev
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                               OneHotEncoder , StringIndexer)
import DataSet.SnpsSelection as s

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.linalg import DenseVector
from pyspark.mllib.evaluation import MulticlassMetrics

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LinearSVC

from pyspark.ml.classification import NaiveBayes

from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel


import math
import time


def writeCoef(path,snpsIds,sc,idToName,name = None):
        
    if not name:
        print("give a name to file")
        return
        
    p = path + name  + " ( " + time.strftime("%d-%m-%Y") + " ).txt "  
    
    i=1
    while os.path.exists(p):
            
        p = path + name  + " ( " + time.strftime("%d-%m-%Y") + " ) " + '_' + str(i)+".txt"
        i += 1
        
    snps = []
    for i in range(len(snpsIds)):
        s = snpsIds[i]
        #snps.append(idToName[s])
        snps.append(s)
            
    print("snpsIds = ",len(snpsIds))
    print("idToName = ",len(idToName))
        
    write = open(p,'w')
    for i in range(len(snps)):
            
        write.write(str(snps[i])+'\t'+str(sc[i])+'\n')
            
    write.close()


def calcCoeff(path,coefs, columns):
    
    ids = {}
    ids['coef']={}
   
    print("before = ",len(set(coefs)))
    #for i in range(len(coefs)):
     #   coefs[i] = abs(coefs[i])

    print("after = ",len(set(coefs)))

    idToName = {}
    nameToId = {}

    for i in range(len(coefs)):
        nameToId[coefs[i]] = []

    for i in range(len(coefs)):
        nameToId[coefs[i]].append(i)
        idToName[i] = coefs[i]


    ids['coef']['nameToId'] = nameToId
    ids['coef']['idToName'] = idToName


    sc = sorted(coefs,reverse=True)

    top_30 = []

    for i in range(30):

        snp = ids['coef']['nameToId'][sc[i]][0]
        ids['coef']['nameToId'][sc[i]].remove(snp)
        top_30.append(columns[snp])
     #   top_30.append(snp)

    #snpReduc['low'] = top_30
    writeCoef(path,top_30,sc,columns, 'mycutoffabs')
    
    
def showMetrics(c,value=1):
    
    predictionAndLabels = c.map(lambda lp: (lp.prediction, float(lp.label)))
    metrics = MulticlassMetrics(predictionAndLabels)

    print("confusion matrix = ", metrics.confusionMatrix().toArray())

    print("accuracy = ", metrics.accuracy)
    print("recal = ", metrics.recall(value))
    print("precision = ", metrics.precision(value))
    print("f1 = ", metrics.fMeasure(1.0))
    
    
def showMyMetric(results,v1 = 1,v2 = 0):
    
    if v1 == v2:
        print("wrong values!!!v1 is the same with v2!!!!")
       
    
    r = results.rdd
    r3 = r.collect()

    same0 = 0
    same1 = 0
    sum0 = 0
    sum1 = 0
    sumall = 0
    for i in r3:

        if i[v1] == 0:
            sum0 += 1
            if i[v2] == 0:
                same0 += 1
        elif i[v1] == 1:
            sum1 += 1
            if i[v2] == 1:
                same1 += 1
        sumall += 1

    print('sum0 = ', sum0)
    print('sum1 = ', sum1)
    print('same0 = ', same0)
    print('same1 = ', same1)
    print('all = ', sumall)
    print('all2 = ', sum0+sum1)

In [ ]:
from pyspark import SparkConf, SparkContext
conf = (SparkConf()
    .set("spark.driver.maxResultSize", "20g")
    .set('spark_executor_cores',"3")
    .set('spark.graphx.pregel.checkpointInterval','-1')
    .set('spark.network.timeout','100000000')
    .set('spark.executor.heartbeatInterval','10000000'))

#spark = SparkSession.builder.appName('melanoma').getOrCreate()
sc = SparkContext(conf=conf)
spark = SQLContext(sc)

In [ ]:
#spark.stop()

# Tests

In [ ]:
path = '/media/antonis/red/newdata/maf = 0.05/pvalue = 0.001/'

#pathSnp = '/media/antonis/red/newSet/maf/maf = 0.05/assoc/pvalue = 0.001/snpCodeTest.csv'
#pathSnp = '/media/antonis/red/newSet/maf/maf = 0.05/assoc/pvalue = 0.01/snp2.txt'
#pathSnp = '/media/antonis/red/newdata/maf = 0.05/pvalue = 0.001/snp2.txt'
pathSnp = '/media/antonis/red/newdata/maf = 0.05/pvalue = 0.001/snpCodeTest.csv'

#pathSnp = '/media/antonis/Antonis_Moulopoulos/newSet/pvalue = 0.001/snp1.txt'
#pathSnp = '/media/antonis/Antonis_Moulopoulos/newdata/maf = 0.05/pvalue = 0.001/snp2.txt'


data = spark.read.option("maxColumns", 80000).csv(pathSnp,inferSchema=True,header=True)
data=data.withColumnRenamed('TARGET','label')

In [ ]:
features = []
for i in data.columns:
    if 'rs' not in i and i !='label':
        features.append(i)
print(len(features))
#print((features))

In [ ]:
#data.select('patients','label').show(10)

In [ ]:
from pyspark.ml.linalg import DenseVector

d = data.drop('patients')
for i in features:
    d = d.drop(i)
print("columns = ",len(d.columns))
li = len(d.columns)
input_data = d.rdd.map(lambda x: (x[li-1], DenseVector(x[:li-1])))

df = spark.createDataFrame(input_data, ["label", "features"])

In [ ]:
df.show(10)

# Correlation

In [ ]:
from pyspark.ml.stat import Correlation
final_data1 = df.select('features')
corr = Correlation.corr(final_data1, "features")
corr = corr.head()[0].toArray()

In [ ]:
snpsRed = []
snpsRed = s.lowCorrelation(corr, threshold=0.7, up=100, down=99.5)#oso megalytero threshold toso perissotero omoia einai 

In [ ]:

d = data.drop('patients')
features = []

for i in data.columns:
    if 'rs' in i and i !='label':
        features.append(i)
print(len(features))

#features = d.columns

print("len features1 = ",len(features))

snpsRed1 = []
for i in snpsRed:
    snpsRed1.append(features[i])

snpsRed1.append('label')
    
features = []
for i in data.columns:
    if i not in snpsRed1:
        features.append(i)
print("len features2 = ",len(features))

#for i in features:
#    d = d.drop(i)
d = d.drop(*features)
print("columns = ",len(d.columns))
li = len(d.columns)
input_data = d.rdd.map(lambda x: (x[li-1], DenseVector(x[:li-1])))

df = spark.createDataFrame(input_data, ["label", "features"])

#assempler = VectorAssembler(inputCols=snpsRed1,outputCol='snpsRed')
#output = assempler.transform(data)
#final_data = output.select('snpsRed','label')
#final_data.show(10)

# Balanced Label1 to train

In [ ]:
train1 = df.filter("label == 1")
train0 = df.filter("label == 0")

label1 = train1.count()
label0 = train0.count()

l1 = label1 + (label1 * 0.1)

x = ((label1/2)*100) / label0

x = x / 100


tr1,tr11 = train1.randomSplit([0.5,0.5],seed=11)
tr0,tr00 = train0.randomSplit([x,1-x],seed=11)

train0 = tr0.union(tr1)
test_data = tr00.union(tr11)

t0 = train0.count()
t1 = test_data.count()

print("label0 = ",label0)
print('label1 = ', label1)
print("l1 = ",l1)
print('train = ', t0)
print('test = ', t1)
print('all = ', t0 + t1)
print('df = ',df.count())

# UnBalanced Label1 to train

In [ ]:
train_data,test_data = df.randomSplit([0.9,0.1],seed=11)

label1 = train_data.filter("label == 1").count()
label0 = train_data.filter("label == 0").count()

x = (label1*100) / label0

x = x / 100

print("label1 = ", label1)
print("label0 = ", label0)
print("x = ", x)

train0,t2 = train_data.filter('label == 0').randomSplit([x,1-x],seed = 11)
train1 = train_data.filter("label == 1")

train0 = train0.union(train1)

In [ ]:
print(train0.filter('label == 0').count())
print(train0.filter('label == 1').count())
print(train0.count())
print(len(d.columns))


# dokimh 

In [ ]:
'''t0,t = data.filter('label == 0').randomSplit([0.001,1-0.001],seed=11)
t1 = data.filter('label == 1')

train_dok = t0.union(t1)

features = []
for i in train_dok.columns:
    if 'rs' not in i and i !='label':
        if i == 'label':
            print('label')
        features.append(i)
print(len(features))

train_dok = train_dok.drop('patients')
train_dok = train_dok.drop(*features)

print("columns = ",len(train_dok.columns))
li = len(train_dok.columns)

print("last column = ", train_dok.columns[li-1])

input_data = train_dok.rdd.map(lambda x: (x[li-1], DenseVector(x[:li-1])))
df = spark.createDataFrame(input_data, ["label", "features"])

print("df count = ", df.count())

'''

In [ ]:
from pyspark.ml.stat import Correlation
final_data1 = df.select('features')
corr = Correlation.corr(final_data1, "features")
corr = corr.head()[0].toArray()

In [ ]:
snpsRed = []
snpsRed = s.lowCorrelation(corr, threshold=0.7, up=100, down=100)#oso megalytero threshold toso perissotero omoia einai 

In [ ]:

features = []

for i in range(0,len(train_dok.columns)):
    if 'rs' in train_dok.columns[i] and train_dok.columns[i] !='label':
        features.append(train_dok.columns[i])

print("len features1 = ",len(features))

snpsRed1 = []
for i in snpsRed:
    snpsRed1.append(features[i])

snpsRed1.append('label')

print('len snpsRed1 = ', len(snpsRed1))
    
features = []
for i in range(0,len(data.columns)):
    if data.columns[i] not in snpsRed1:
        features.append(data.columns[i])
print("len features2 = ",len(features))

d = data.drop(*features)
d = d.drop('patients')
print("columns = ",len(d.columns))
li = len(d.columns)

print('first column = ', d.columns[0])
print('last column = ', d.columns[li - 1])

input_data = d.rdd.map(lambda x: (x[li-1], DenseVector(x[:li-1])))
df = spark.createDataFrame(input_data, ["label", "features"])



In [ ]:
train1 = df.filter("label == 1")
train0 = df.filter("label == 0")

label1 = train1.count()
label0 = train0.count()

l1 = label1 + (label1 * 0.1)

x = ((label1/2)*100) / label0

x = x / 100


tr1,tr11 = train1.randomSplit([0.5,0.5],seed=11)
tr0,tr00 = train0.randomSplit([x,1-x],seed=11)

train0 = tr0.union(tr1)
test_data = tr00.union(tr11)

t0 = train0.count()
t1 = test_data.count()

print("label0 = ",label0)
print('label1 = ', label1)
print("l1 = ",l1)
print('train = ', t0)
print('test = ', t1)
print('all = ', t0 + t1)
print('df = ',df.count())

# telos dokimhs

# Linear Regression

In [ ]:
log_reg = LogisticRegression(featuresCol='features',labelCol='label',maxIter=10)
lr_model = log_reg.fit(train0)

In [ ]:

results = lr_model.transform(test_data)
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluate = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='label')
AUC = evaluate.evaluate(results)
AUC

In [ ]:
'''Param for metric name in evaluation. Supports: - "rmse" (default): root mean squared error - 
"mse": mean squared error - "r2": R^2^ metric - "mae": mean absolute error metricName='mae' '''

pipeline = Pipeline(stages=[log_reg])
modelEvaluator=RegressionEvaluator(predictionCol='prediction', labelCol='label',metricName='mae')
paramGrid = ParamGridBuilder().build()

crossval = CrossValidator(estimator=log_reg,
                          estimatorParamMaps=paramGrid,
                          evaluator=modelEvaluator,
                          numFolds=6)

cvModel = crossval.fit(train0)
cross_results = cvModel.transform(test_data)
results = cross_results.select('prediction','label')
#results.show(10)

In [ ]:
#print(len(d.columns))
#print(d.columns[len(d.columns)-1])
calcCoeff(path,lr_model.coefficientMatrix.toArray()[0],d.columns)

In [ ]:
showMetrics(cross_results.rdd,value=1)
showMyMetric(results,v1 = 1,v2 = 0)

In [ ]:
lr_model.coefficientMatrix.toArray()[0].shape

# RFR

In [ ]:
rf = RandomForestClassifier(featuresCol='features',labelCol='label', numClasses=2, categoricalFeaturesInfo={},
                                     numTrees=50, featureSubsetStrategy="auto",
                                     impurity='gini', maxDepth=4, maxBins=32)
rf_model = rf.fit(train0)

In [ ]:
results = rf_model.transform(test_data)
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluate = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='label')
AUC = evaluate.evaluate(results)
AUC

In [ ]:

modelEvaluator=RegressionEvaluator(predictionCol='prediction', labelCol='label')
paramGrid = ParamGridBuilder().build()

crossval = CrossValidator(estimator=rf,
                          estimatorParamMaps=paramGrid,
                          evaluator=modelEvaluator,
                          numFolds=4)

cvModel = crossval.fit(train0)
cross_results = cvModel.transform(test_data)
results = cross_results.select('prediction','label')
#results.show(10)

In [ ]:
showMetrics(cross_results.rdd,value=1)
showMyMetric(results,v1 = 1,v2 = 0)

# SVM 

In [ ]:
#lsvc = LinearSVC(maxIter=10, regParam=0.1)
lsvc = LinearSVC(featuresCol='features',labelCol='label',maxIter=10)
lsvcModel = lsvc.fit(train0)

In [ ]:
results = lsvcModel.transform(test_data)
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluate = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='label')
AUC = evaluate.evaluate(results)
AUC

In [ ]:
modelEvaluator=RegressionEvaluator(predictionCol='prediction', labelCol='label')
paramGrid = ParamGridBuilder().build()

crossval = CrossValidator(estimator=lsvcModel,
                          estimatorParamMaps=paramGrid,
                          evaluator=modelEvaluator,
                          numFolds=4)

cvModel = crossval.fit(train0)
cross_results = cvModel.transform(test_data)
results = cross_results.select('prediction','label')
#results.show(10)

In [ ]:
showMetrics(cross_results.rdd,value=1)
showMyMetric(results,v1 = 1,v2 = 0)

In [ ]:
#calcCoeff(path,lr_model.coefficientMatrix.toArray()[0],d.columns)

# Bernoulli

In [ ]:
nb = NaiveBayes(predictionCol='prediction', labelCol='label',smoothing=1.0, modelType="multinomial")
nbg = nb.fit(train0)

In [ ]:
results = nbg.transform(test_data)
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluate = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='label')
AUC = evaluate.evaluate(results)
AUC

In [ ]:
modelEvaluator=RegressionEvaluator(predictionCol='prediction', labelCol='label')
paramGrid = ParamGridBuilder().build()

crossval = CrossValidator(estimator=nbg,
                          estimatorParamMaps=paramGrid,
                          evaluator=modelEvaluator,
                          numFolds=4)

cvModel = crossval.fit(train0)
cross_results = cvModel.transform(test_data)
results = cross_results.select('prediction','label')
#results.show(10)

In [ ]:
showMetrics(cross_results.rdd,value=1)
showMyMetric(results,v1 = 1,v2 = 0)

# Tree

In [ ]:
t = train.rdd
tr =  t.map(lambda lp: (int(lp.label), int(lp.features)))
treeD = DecisionTree.trainClassifier(tr, numClasses=2, categoricalFeaturesInfo={},
                                     impurity='gini', maxDepth=5, maxBins=32)


tree = treeD.fit(train0)

In [ ]:
results = tree.transform(test_data)
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluate = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='label')
AUC = evaluate.evaluate(results)
AUC

In [ ]:
modelEvaluator=RegressionEvaluator(predictionCol='prediction', labelCol='label')
paramGrid = ParamGridBuilder().build()

crossval = CrossValidator(estimator=tree,
                          estimatorParamMaps=paramGrid,
                          evaluator=modelEvaluator,
                          numFolds=4)

cvModel = crossval.fit(train0)
cross_results = cvModel.transform(test_data)
results = cross_results.select('prediction','label')
#results.show(10)

In [ ]:
showMetrics(cross_results.rdd,value=1)
showMyMetric(results,v1 = 1,v2 = 0)


In [ ]:
modelFile = path+'treeRules.txt'
f = open(modelFile,"w") 
f.write(crossval.toDebugString())
f.close()

# Gradient-Boosted Trees (GBTs)

In [ ]:
modelGBT = GradientBoostedTrees.trainClassifier(train0,
                                                                     categoricalFeaturesInfo={}, numIterations=10)

gbt = treeD.fit(train0)

In [ ]:
results = gbt.transform(test_data)
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluate = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='label')
AUC = evaluate.evaluate(results)
AUC

In [ ]:
modelEvaluator=RegressionEvaluator(predictionCol='prediction', labelCol='label')
paramGrid = ParamGridBuilder().build()

crossval = CrossValidator(estimator=gbt,
                          estimatorParamMaps=paramGrid,
                          evaluator=modelEvaluator,
                          numFolds=4)

cvModel = crossval.fit(train0)
cross_results = cvModel.transform(test_data)
results = cross_results.select('prediction','label')
#results.show(10)

In [ ]:
showMetrics(cross_results.rdd,value=1)
showMyMetric(results,v1 = 1,v2 = 0)